<a href="https://colab.research.google.com/github/mannam95/Deep_Learning_Programming/blob/main/Assignment6/Assignment_6_RNN_Part2_Srinath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team Assignment


1.   Srinath Mannam (229750)
2.   Meghana Rao (234907)
3.   Govind Shukla (235192)

# import statements

In [1]:
import os
import sys
import numpy as np
from datetime import datetime
import tensorflow as tf
from keras.preprocessing import sequence

# Change the directory

In [14]:
working_directory = '/content/drive/My Drive/Colab Notebooks/OVGU/Deep_Learning/06_Assignment'
def colabDrive():
    from google.colab import drive
    drive.mount("/content/drive")

    if os.getcwd() !=  working_directory:
      os.chdir(working_directory)
    print(os.getcwd())

In [15]:
colabDrive()

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/OVGU/Deep_Learning/06_Assignment


# Prepare the data

## Load the dataset and remove infrequent words

In [2]:
num_words = 40000
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=num_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

print('Loaded dataset with {} training samples, {} test samples'.format(len(train_sequences), len(test_sequences)))

17473536/17464789 [==============================] - 1s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


## to restore words, load the word-to-index mapping and look at one example

In [3]:
word2id = tf.keras.datasets.imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in train_sequences[6]])
print('---label---')
print(train_sequences[6])
print('---label---')
print(train_labels[6])

1654784/1641221 [==============================] - 0s 0us/step
---review with words---
['the', 'boiled', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'murdering', 'naschy', 'br', 'villain', 'council', 'suggestion', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'echoed', 'concentrates', 'concept', 'issue', 'skeptical', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'rocketed', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', "captain's", 'starship', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'originals', 'things', 'is', 'far', 'this', 'make', 'mistakes', "kevin's", 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'dose', 'movies', 'get

## Maximum review length and minimum review length.

In [4]:
print('Maximum review length: {}'.format(len(max((train_sequences + test_sequences), key=len))))

Maximum review length: 2697


In [5]:
print('Minimum review length: {}'.format(len(min((train_sequences + test_sequences), key=len))))

Minimum review length: 70


## Generator to create different shaped train,test dataset

In [6]:
def gen_Train():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label
      
def gen_Test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

## Create training, test dataset tensors

In [7]:
def createData(bucketFlag):
  train_data = tf.data.Dataset.from_generator(gen_Train, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))

  test_data = tf.data.Dataset.from_generator(gen_Test, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))
  
  # Do bucket padding if the flag is true or do normal padding
  if bucketFlag:
    buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    bucket_batch_size = [32] * (len(buckets) + 1)
    train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                      bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size, drop_remainder=True)
    test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                      bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size, drop_remainder=True)
    return (train_data, test_data)

  train_data = train_data.padded_batch(32, drop_remainder=True)
  test_data = test_data.padded_batch(32, drop_remainder=True)
  return (train_data, test_data)

In [8]:
(train_data, test_data) = createData(True)

In [9]:
print("Train sequences Batches")
for step, (sequence, label) in enumerate(train_data):
    print(sequence.shape, label.shape)

    if step is 3:
      break

print("\nTest sequences Batches")
for step, (sequence, label) in enumerate(test_data):
    print(sequence.shape, label.shape)

    if step is 3:
      break

Train sequences Batches
(32, 147) (32,)
(32, 198) (32,)
(32, 1038) (32,)
(32, 240) (32,)

Test sequences Batches
(32, 141) (32,)
(32, 197) (32,)
(32, 149) (32,)
(32, 249) (32,)


# Generic Functions

## Training Loop

In [10]:
def model_Train(curr_Model, filePath, EPOCHS=5,):
  temp_Model = curr_Model

  temp_Model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
  
  temp_Model.fit(train_data, epochs=EPOCHS)

  temp_Model.save('saved_models/' + filePath)

## Testing Loop

In [17]:
def model_Test(filePath):

  loaded_model = tf.keras.models.load_model(filePath)

  print("Model Summary")
  loaded_model.summary()

  print("Model Train Accuracy")
  loaded_model.evaluate(train_data)

  print("\nModel Test Accuracy")
  loaded_model.evaluate(test_data)

  print("\n\n")

# Models

## Training

### RNN

In [ ]:
model_rnn = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.RNN(tf.keras.layers.SimpleRNNCell(12)),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_Train(model_rnn, filePath='model_rnn_epochs10_bucketPad_V1.h5', EPOCHS=10)

### LSTM

In [ ]:
model_lstm = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.LSTM(12),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_Train(model_lstm, filePath='model_lstm_epochs10_bucketPad_V1.h5', EPOCHS=10)

### Bi-directional LSTM

In [ ]:
model_blstm = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(12)),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_Train(model_blstm, filePath='model_blstm_epochs10_bucketPad_V1.h5', EPOCHS=10)

## Testing

In [19]:
models_Path = {
    'rnn_models': ['saved_models/model_rnn_epochs10_bucketPad_V1.h5',
                   'saved_models/model_rnn_epochs10_bucketPad_V2.h5'],
    'lstm_models': ['saved_models/model_lstm_epochs10_bucketPad_V1.h5'],
    'blstm_models': ['saved_models/model_blstm_epochs10_bucketPad_V1.h5' ]
}

### Summary


*   Below are the training and testing accuracy for all different model configurations.
*   For some weird reason RNN is very slow in colab it took apprx 40 minutes and 10 mins in local PC.
*   LSTM, Bi-directional-LSTM took around 4 minutes in colab.

Note:- The models are trained in different colab accounts. In order to summarise easily, I have saved the trained models and getting the metrics in below. Probably because of this reason in the training section the output cells might be empty.



In [20]:
for key in models_Path:
  print("----------------------------------Model Type----------------------------------------\n")
  print("Currently Testing the {} \n".format(key))
  for ind, val in enumerate(models_Path[key]):
    model_Test(filePath=val)

----------------------------------Model Type----------------------------------------

Currently Testing the rnn_models 

Model Summary
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1280000   
                                                                 
 rnn (RNN)                   (None, 12)                540       
                                                                 
 dense (Dense)               (None, 1)                 13        
                                                                 
Total params: 1,280,553
Trainable params: 1,280,553
Non-trainable params: 0
_________________________________________________________________
Model Train Accuracy
775/775 [==============================] - 26s 33ms/step - loss: 0.1076 - accuracy: 0.9858

Model Test Accuracy
776/776 [==============================] -